#  3^(0.5) division method proposed by Leif Kobbelt  in libgl

## Import Related Dependencies

In [ ]:
import igl
import math
import scipy as sp
import numpy as np
from meshplot import plot, subplot, interact

In [ ]:
# Read the files, named variable sphere but can be any shape

v_sphere, f_sphere = igl.read_triangle_mesh("data/sphere.obj")

# Step 1: Add new  Vertex/Faces

midpoints = []
faces = []
midpoint_index = len(v_sphere)

# Iterate through the vertex, calculate the midpoints
for face in f_sphere:
    vertex1 = face[0]
    vertex2 = face[1]
    vertex3 = face[2]
    vertex_m = (v_sphere[vertex1]+v_sphere[vertex2]+v_sphere[vertex3])/3
    midpoints.append(vertex_m)
    face1 = [vertex1, vertex2, midpoint_index]
    face2 = [vertex2, vertex3, midpoint_index]
    face3 = [vertex3, vertex1, midpoint_index]
    midpoint_index += 1
    faces.append(face1)
    faces.append(face2)
    faces.append(face3)


m_v_sphere = np.array(midpoints)
new_faces = np.array(faces)


# New list of vertices and faces
v_sphere_new = np.append(v_sphere, m_v_sphere).reshape(-1, 3)
f_sphere_new = np.append(f_sphere, new_faces).reshape(-1, 3)

# Step 2: Move the positions of the original

adj_list = igl.adjacency_list(f_sphere)
new_position = []
for i in range(len(v_sphere)):
    # Calculate an
    an = (4-2*np.cos(2*math.pi/len(adj_list[i])))/9

    # Calculate the sum of adj vertices
    adj_positions = np.array([v_sphere[j] for j in adj_list[i]])
    adj_sum = np.sum(adj_positions, axis=0)

    # Calculate new position p
    p = (1-an)*v_sphere[i]+an/len(adj_list[i])*adj_sum

    new_position.append(p)

new_positions = np.array(new_position)

v_sub_mesh = np.append(new_positions, m_v_sphere).reshape(-1, 3)

# Step 3: Flip Edges

tt,tti=igl.triangle_triangle_adjacency(new_faces)

# function to find similar two points
def find_outer_triangle(trianglecurrent,triangle1, triangle2,triangle3):
    if(trianglecurrent[2]!=triangle1[2]):
        return triangle1
    elif(trianglecurrent[2]!=triangle2[2]):
        return triangle2
    elif(trianglecurrent[2]!=triangle3[2]):
        return triangle3


f_final = []
for i in range(len(new_faces)):
    trianglecurrent = new_faces[i]
    midpointcurrent = trianglecurrent[2]
    triangle1=new_faces[tt[i][0]]
    triangle2=new_faces[tt[i][1]]
    triangle3=new_faces[tt[i][2]]
    outertriangle=find_outer_triangle(trianglecurrent,triangle1, triangle2,triangle3)
    midpointouter=outertriangle[2]
    face1=[midpointcurrent,midpointouter,trianglecurrent[0]]
    face2=[midpointcurrent,midpointouter,trianglecurrent[1]]
    f_final.append(face1)
    f_final.append(face2)

f_final = np.array(f_final)


plot(v_sphere,f_sphere, shading={"wireframe": True})
plot(v_sub_mesh, f_final, shading={"wireframe": True})


